# Latihan Regresi

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

# Mengunduh  dataset California Housing
X, y = fetch_california_housing(return_X_y=True)

# Membagi dataset menjadi training set dan testing set (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Melakukan scaling pada data (penting untuk regresi)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Shape of training data:", X_train.shape)
print("Shape of testing data:", X_test.shape)

Shape of training data: (14448, 8)
Shape of testing data: (6192, 8)


output diatas memiliki 14.448 data latih dan 6192 data uji. kemudian mari kita latih model machine learning agar dapat memprediksi nilai kontinu karena kita akan membuat sebuah model regresi


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Inisialisasi model Random Forest Regressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
 
# Evaluasi awal model tanpa tuning
y_pred = rf.predict(X_test)
initial_mse = mean_squared_error(y_test, y_pred)
print(f"Initial MSE on test set (without tuning): {initial_mse:.2f}")

Initial MSE on test set (without tuning): 0.26


## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV


start_time = time.time() # mencatat waktu mulai

# Definisikan parameter grid untuk Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Inisialisasi Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=1, verbose=2)
grid_search.fit(X_train, y_train)

# output hasil terbaik 
print(f'Best parameters (Grid Search): {grid_search.best_params_}')
best_rf_grid = grid_search.best_estimator_

# Evaluasi peforma model setelah grid search
y_pred_grid = best_rf_grid.predict(X_test)
grid_search_mse = mean_squared_error(y_test, y_pred_grid)
print(f'MSE after Grid Search: {grid_search_mse:.2f}')
end_time = time.time() # mencatat waktu selesai
execution_time = end_time - start_time # menghitung selisih waktu
print(f'Waktu Eksekusi: {execution_time:.4f} detik')

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.5s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   6.9s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   7.7s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   9.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=  12.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=  12.3s
[

## Random Search


In [4]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

start_time = time.time()

# Definisiakn ruang pencarian untuk Random Search 
param_dist = {
    'n_estimators': np.arange(100, 500, 100),
    'max_depth': [None] + list(np.arange(10, 50, 10)),
    'min_samples_split': np.arange(2, 11, 2),
    'min_samples_leaf': np.arange(1, 5),
    'bootstrap': [True, False]
}

# Inisialisasi RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=5, cv=3, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

# output hasil terbaik 
print(f'Best Parameters (Random Search): {random_search.best_params_}')
best_rf_random = random_search.best_estimator_

# Evaluasi peforma model setelah Random Search 
y_pred_random = best_rf_random.predict(X_test)
random_search_mse = mean_squared_error(y_test, y_pred_random)
print(f'MSE after Grid Search: {random_search_mse:.2f}')
end_time = time.time()
execution_time = end_time - start_time
print(f'Waktu eksekusi: {execution_time:.4f} detik')

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best Parameters (Random Search): {'n_estimators': np.int64(300), 'min_samples_split': np.int64(6), 'min_samples_leaf': np.int64(2), 'max_depth': np.int64(30), 'bootstrap': True}
MSE after Grid Search: 0.25
Waktu eksekusi: 69.7461 detik


## Bayesian Optimization


In [4]:
from skopt import BayesSearchCV
 
start_time = time.time()  # Mencatat waktu mulai
 
# Definisikan ruang pencarian untuk Bayesian Optimization
param_space = {
    'n_estimators': (100, 500),
    'max_depth': (10, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4),
    'bootstrap': [True, False]
}
 
# Inisialisasi BayesSearchCV
bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=32, cv=3, n_jobs=1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)
 
# Output hasil terbaik
print(f"Best parameters (Bayesian Optimization): {bayes_search.best_params_}")
best_rf_bayes = bayes_search.best_estimator_
 
# Evaluasi performa model setelah Random Search
y_pred_bayes = best_rf_bayes.predict(X_test)
bayes_mse = mean_squared_error(y_test, y_pred_bayes)
print(f"MSE after Grid Search: {bayes_mse:.2f}")
end_time = time.time()  # Mencatat waktu selesai
execution_time = end_time - start_time  # Menghitung selisih waktu
print(f"Waktu eksekusi: {execution_time:.4f} detik")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=True, max_depth=39, min_samples_leaf=4, min_samples_split=5, n_estimators=368; total time=  15.5s
[CV] END bootstrap=True, max_depth=39, min_samples_leaf=4, min_samples_split=5, n_estimators=368; total time=  15.4s
[CV] END bootstrap=True, max_depth=39, min_samples_leaf=4, min_samples_split=5, n_estimators=368; total time=  15.6s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=False, max_depth=45, min_samples_leaf=2, min_samples_split=10, n_estimators=446; total time=  31.2s
[CV] END bootstrap=False, max_depth=45, min_samples_leaf=2, min_samples_split=10, n_estimators=446; total time=  29.6s
[CV] END bootstrap=False, max_depth=45, min_samples_leaf=2, min_samples_split=10, n_estimators=446; total time=  29.5s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=True, max_depth=47, min_samples_leaf=1, min_samples_split=5, n_estimators=175; total time=   9.1